In [1]:
from threading import *
from datetime import datetime, timedelta
import socket
import random
import time

SWS = 8
buffer = [''] * SWS
buffer_index = 0
active_windows = [0] * SWS
timers = [-1] * SWS
last_ack = -1
flag = False
end_of_file = False
frame_count = 0
end_of_program = False

"""
To check this checksum code try: hex(int(checksum(bin(61956)[2:] + bin(62709)[2:] + bin(63223)[2:])[2:], 2))
expected output : '0x220d'

"""

def checksum(data):
    checksum = '0' * 16
    data_packets = [data[i:i+16] for i in range(len(data))[::16]]
    for i in data_packets:
        i = int(i , 2)
        sum_int = int(checksum, 2)
        temp = bin(i + sum_int)
        while (len(temp) > 18):
            temp = bin(int(temp[3:], 2) + 1)
        checksum = temp[2:]
    checksum = '0' * (16 - len(checksum)) + checksum
    return ''.join(['0' if i == '1' else '1' for i in checksum])

def create_frame(data, seq_no):
    return '0' * (3 - len(bin(seq_no)[2:])) + bin(seq_no)[2:] + str(data) + str(checksum(data))

def process(bit_string):
    global frame_count
    frame_count += 1
    len_string = len(bit_string)
    rand = frame_count % 10 == 0
    changed_string = bit_string
    if rand:
        try:
            rand_index = random.randint(1, len_string - 1)
            changed_string = ''.join([str(int(bit_string[i]) ^ 1) if i == rand_index else bit_string[i] for i in range(len(bit_string))])
        except:
            pass
    return changed_string

#l = Lock()

def give_time():
    
    global SWS
    global buffer
    global buffer_index
    global active_windows
    global timers
    global last_ack
    global flag
    global end_of_file
    global end_of_program
    
    while True:
        if end_of_program:
            break;
        temp = datetime.now()
        try:
            k = (last_ack + 1) % SWS
            if temp >= timers[k]:
                flag = True
        except Exception:
            continue

def sender(filename, n, c):
    
    global SWS
    global buffer
    global buffer_index
    global active_windows
    global timers
    global last_ack
    global flag
    global end_of_file
    global eng_of_program
    
    with open(filename) as f:
        frame_count = 0
        while True:
            if end_of_program:
                break;
                
            if not end_of_file:
                frame = f.read(n)
                frame_count += 1
                if not frame:
                    #l.acquire()
                    print('End of file')
                    #l.release()
                    end_of_file = True
                bframe = ""
                for i in frame:
                    bframe += '0' * (8 - len(bin(ord(i))[2:])) + bin(ord(i))[2:]

                data_to_send = create_frame(bframe, buffer_index)
            else:
                data_to_send = None
            while True:
                if flag:
                    #l.acquire()
                    print("Frames being discarded: {0} to {1}".format((last_ack+1)%SWS, buffer_index))
                    #l.release()
                    buffer_index = (last_ack + 1) % SWS
                    flag = False
                    
                #if active_windows != [1] * SWS:
                try:
                    time.sleep(0.01)
                    active_windows[buffer_index] = 1
                    if buffer[buffer_index] == '' and data_to_send:
                        buffer[buffer_index] = data_to_send
                    to_send = buffer[buffer_index][:3] + process(buffer[buffer_index][3:-16]) + buffer[buffer_index][-16:]
                    c.send(to_send.encode())
                    temp = datetime.now() + timedelta(milliseconds = 50)
                    timers[buffer_index] = temp
                    #l.acquire()
                    print(active_windows)
                    #l.acquire()
                    buffer_index += 1
                    buffer_index %= SWS
                except Exception as e:
                    print('Sender error')
                    print("Error:", e)
                break

def reciever(c):
    
    global SWS
    global buffer
    global buffer_index
    global active_windows
    global timers
    global last_ack
    global flag
    global end_of_file
    global end_of_program

    while True:
        if end_of_file and active_windows == [0] * SWS:
            end_of_program = True
            break;
        try:
            data = c.recv(1024)
            if len(data) != 0:
                #l.acquire()
                print("Ack Recieved: ", data.decode())
                #l.release()
                last_ack = (last_ack + 1) % SWS
                buffer[last_ack] = ''
                timers[last_ack] = -1
                active_windows[last_ack] = 0
                print(active_windows)
            else:
                c.close()
                print("Connection closed by client.")
                break
        except Exception as e:
            print('Reciever Error:', e)
            return

In [2]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print('Socket created')
port = 12347
s.bind(('', port))
s.listen(5)
print('socket listening')
n = 16
filename = 'data.txt'

try:
    c, addr = s.accept()
    print('Connected with', addr)
    try:
        t1 = Thread(target = sender, args = (filename, n, c))
        t2 = Thread(target = reciever, args = (c,))
        t3 = Thread(target = give_time)
        t1.start()
        t2.start()
        t3.start()
        t1.join()
        t2.join()
        t3.join()
    except Exception as e:
        print(str(e))
        s.close()
except Exception as e:
    c.close()
    print('closed connection with ' + str(addr))
    s.close()

Socket created
socket listening
Connected with ('127.0.0.1', 49218)
[1, 0, 0, 0, 0, 0, 0, 0]
Ack Recieved:  ack0
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
Ack Recieved:  ack1
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
Ack Recieved:  ack2
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
Ack Recieved:  ack3
[0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
Ack Recieved:  ack4
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
Ack Recieved:  ack5
[0, 0, 0, 0, 0, 0, 1, 0]
Ack Recieved:  ack6
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]Ack Recieved: 
 ack7
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]Ack Recieved: 
 ack0
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0]
Frames being discarded: 1 to 3
[0, 1, 1, 0, 0, 0, 0, 0]Ack Recieved:  ack1
[0, 0, 1, 0, 0, 0, 0, 0]

Frames being discarded: 2 to 2
[0, 0, 1, 0, 0, 0, 0, 0]Ack Recieved: 
Frames being discarded: 2 to 3
 ack2
[0, 0, 1, 0, 0, 0, 0, 0][0, 0, 0, 0, 0, 0, 0